# Web scraping avec Selenium
## Qu'est-ce que le web-scraping?

Le web scraping est une technique permettant d'extraire automatiquement des informations d'Internet à l'aide d'un logiciel qui simule la navigation humaine sur le Web.

## Présentation de Selenium

Le logiciel libre Selenium est un framework pour réaliser des tests automatisés d’applications web. Développé à l’origine pour tester les sites Internet et les applications web, le pilote web Selenium peut également être utilisé pour scraper des sites Internet avec Python. Bien que Selenium ne soit pas codé en Python, les fonctionnalités de ce logiciel sont accessibles depuis Python.

Contrairement à Scrapy ou BeautifulSoup, Selenium n’agit pas au niveau du code source HTML. À la place, la page est chargée dans un navigateur sans interface utilisateur. Le navigateur interprète le code source de la page et génère un *Document Object Model* (DOM). Cette interface standardisée permet de tester les interactions des utilisateurs : de cette façon, il est par exemple possible de simuler des clics et de remplir automatiquement des formulaires. Les modifications de la page qui en résultent peuvent être retrouvées dans le DOM. Par conséquent, un web scraping avec Selenium suit le schéma suivant :

URL → requête HTTP → HTML → Selenium → DOM

Le DOM étant généré de façon dynamique, Selenium permet également de scraper des pages dont les contenus ont été générés avec JavaScript. L’accès aux contenus dynamiques constitue un avantage significatif de Selenium. Pour encore plus de praticité, Selenium peut également être utilisé en association avec Scrapy ou BeautifulSoup. Selenium fournira alors le code source tandis que l’autre outil se chargera du parsing et de l’analyse. Dans ce cas, on aura le schéma suivant :

URL → requête HTTP → HTML → Selenium → DOM → HTML → Scrapy / BeautifulSoup

## Tutoriel Selenium

#### Package Selenium

In [ ]:
!pip install selenium

#### Premiers pas dans le Scraping

Plusieurs sites permettent de se faire la main sur Selenium :

* https://towardsdatascience.com/web-scraping-using-selenium-python-8a60f4cf40ab est le site qui présente l'exemple ci-dessous, vous pouvez le suivre pas à pas facilement.

* https://www.ionos.fr/digitalguide/sites-internet/developpement-web/tutoriel-selenium-webdriver/ s'attarde sur le WebDriver. Et avec quelques exemples pratiques.


#### Exemple :
La première chose à faire lors du scraping d'un site Web est de comprendre la structure du site Web. 

Nous allons scraper *Edmunds.com*, un forum automobile. Ce site Web aide les gens dans leurs décisions d'achat de voiture. Les gens peuvent publier leurs avis sur différentes voitures dans les forums de discussion (très similaire à la façon dont on publie des avis sur Amazon). Nous allons scraper la discussion sur les marques de voitures de luxe d'entrée de gamme.

Environ 5000 commentaires d'utilisateurs différents sur plusieurs pages vont être récoltés. Nous récupérerons l'identifiant de l'utilisateur, la date du commentaire et les commentaires et l'exporterons dans un fichier csv pour toute analyse ultérieure.

   ##### Comment réunir tout cela?
Nous pouvons extraire les identifiants, la date et les commentaires de chaque utilisateur sur une page Web particulière en parcourant tous les identifiants de commentaire.
Vous trouverez ci-dessous l'extrait de code pour extraire tous les commentaires d'une page Web particulière.

In [34]:
#Importing packages
from selenium import webdriver
import pandas as pd

charger le fichier geodriver.exe sous c

In [35]:
driver = webdriver.Firefox(executable_path = "C:\geckodriver.exe")

Une nouvelle fenêtre Firefox doit apparaître. Nous allons scraper les commentaires du fil de discussion du forum Edmunds.com consacré aux prix de la Ford Mustang.

Nous enregisterrons :

* l'id de l'utilisateur,
* la date
* le commentaire

In [36]:
driver.get('https://forums.edmunds.com/discussion/58082/ford/mustang/2020-ford-mustang-lease-deals-and-prices')

Sur Firefox, on lance ensuite l'inspection de la page (Ctrl + Maj + I) On commence par exemple par l'User ID. Nous allons récupérer le XPath (XML path).

//*[@id="Discussion_58082"]/div/div[1]/div[1]/span[1]/a[2]

Selenium a une fonction `find_elements_by_xpath`.

In [42]:
userid_element = driver.find_elements_by_xpath('//*[@id="Discussion_58082"]/div/div[1]/div[1]/span[1]/a[2]')[0]
userid = userid_element.text

In [43]:
userid #On obtient bien le résultat attendu

'Michaell'

'Michaell'

##### Scraper pour des users et commentaires multiples

In [44]:
ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
comment_ids = []
for i in ids:
    comment_ids.append(i.get_attribute('id'))

In [45]:
comments = pd.DataFrame(columns = ['Date','user_id','comments']) #Creation d'un dataframe

In [46]:
for x in comment_ids:
    #Extract dates from for each user on a page
    user_date = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
    date = user_date.get_attribute('title')

    #Extract user ids from each user on a page
    userid_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
    userid = userid_element.text

    #Extract Message for each user on a page
    user_message = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[3]/div/div[1]')[0]
    comment = user_message.text
                                   
    #Adding date, userid and comment for each user in a dataframe    
    comments.loc[len(comments)] = [date,userid,comment]

In [47]:
comments

,Date,user_id,comments
0,"September 12, 2019 10:30AM",mrprince87,Can I please have APR and RV and any incentive...
1,"September 12, 2019 10:41AM",kyfdx,mrprince87 said:\nCan I please have APR and RV...
2,"September 13, 2019 10:55PM",Skcraynor,kyfdx said:\nshow previous quotes\n\n\n\n\n0.4...
3,"September 13, 2019 11:50PM",Michaell,Skcraynor said:\nshow previous quotes\n\n\n\n\...
4,"October 9, 2019 1:58PM",PeteT,May I get APR and RV for 2020 Mustang GT Premi...
5,"October 9, 2019 3:03PM",Michaell,PeteT said:\nMay I get APR and RV for 2020 Mus...
6,"October 10, 2019 9:50AM",PeteT,So I have just learned from the dealer..Their ...
7,"October 10, 2019 10:23AM",kyfdx,PeteT said:\nSo I have just learned from the d...
8,"November 10, 2019 10:05PM",Shariqshaik,Ask your 2020 Mustang eccoboost premium lease ...
9,"November 10, 2019 11:43PM",Michaell,Shariqshaik said:\nAsk your 2020 Mustang eccob...


## Les limites

Ce framework open source ne convient pas à tous les domaines comme le soulignent les développeurs dans leur liste de « Worst Cases » (« pires cas d’utilisation »). Les domaines suivants font notamment partie des cas et des contenus de sites internet que vous ne pouvez pas tester ou enregistrer avec Selenium :

* **Captchas** : les captchas sont une solution bien connue et largement répandue, développée pour protéger les sites internet contre les robots et les spams, qui empêche donc les processus automatisés à l’aide de Selenium. Si vous êtes confronté à des captchas, vous devrez donc désactiver les tests de saisie pendant la phase de test ou le scraping ou les remplacer temporairement par un autre élément.


* **Téléchargements de fichier** : même s’il est possible de lancer le téléchargement de fichiers dans les instances Selenium en simulant le clic sur un lien correspondant, l’API n’affiche pas l’avancée du processus de téléchargement.


* **Code de réponse HTTP** : Selenium comporte certaines faiblesses dans le traitement des codes de statut HTTP. Si nécessaire, vous pouvez toutefois parer à ces inconvénients en ajoutant un proxy.


* **Connexion à des services de prestataires tiers** : qu’il s’agisse de plates-formes de réseaux sociaux, de services sur le cloud ou d’un compte de messagerie, il n’est pas recommandé de procéder à une connexion auprès de prestataires tiers à l’aide d’une session Selenium. En effet, les prestataires de tels services mettent à disposition leurs propres API à des fins de test et les tests ainsi réalisés sont très lourds.


* **Test de performance** : Selenium WebDrive ne convient pas à la réalisation de tests de performance à proprement parler car ce framework n’a simplement pas été conçu pour cela.

In [ ]:
//*[@id="Frame"]